In [1]:
import pickle
import matplotlib.pyplot as plt
import librosa
import numpy as np
import os
import librosa.display
import json
from tqdm import tqdm
import random

In [2]:
random.seed(42)

pickle_path = '/scratch/da3245/datasets/neuro_scans/cnn_training_set.pkl'
#pickle_path = '/scratch/da3245/datasets/neuro_scans/KG127-EXPT_1_SCAN_146.pickle'
with open(pickle_path, 'rb') as f:
    fdata = pickle.load(f)

In [1]:
def func2_diverse(data, name, max_samples):
    temp = data['SOMA']
    events = temp['SPIKE_INTERVAL']
    denoised_trace = temp['DENOISED_TRACE']

    output_dir = f"/scratch/cpk286/datasets/data_original/{name}/"
    os.makedirs(output_dir, exist_ok=True)

    mask_json_output_path = os.path.join(output_dir, "mask.json")
    feature_output_path = os.path.join(output_dir, "features.npy")
    trace_json_output_path = os.path.join(output_dir, "trace.json")

    mask = {}
    all_features = []

    sr = 3328
    n_fft = 12
    hop_length = 1
    n_mels = 7

    mel_spectrogram = librosa.feature.melspectrogram(
        y=denoised_trace,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        power=2
    )
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # print(mel_spectrogram_db.shape)
    features = mel_spectrogram_db.T

    labels = events
    # print(labels)

    

    # subsample the per combination limit 
    selected_indices = random.sample(
        list(range(len(labels))),
        max_samples
    )

    # Save the features and labels
    for idx, i in enumerate(selected_indices):
        mask[f"sample_{idx + 1}"] = labels[i]
        all_features.append(features[i])

    # print(mask)

    # Save results
    with open(mask_json_output_path, "w") as json_file:
        json.dump(mask, json_file, indent=1)

    with open(trace_json_output_path, "w") as json_file:
        json.dump(list(denoised_trace), json_file, indent=1)


    all_features = np.array(all_features)
    np.save(feature_output_path, all_features)
    # print(f"Saved features to {feature_output_path}")


total_combinations = sum(len(fdata[expt]) for expt in fdata.keys())
total_samples = 100000 # this controls dataset size
samples_per_combination = total_samples // total_combinations
print(samples_per_combination)
# Process data
for expt in tqdm(fdata.keys()):
    for scan in fdata[expt]:
        data = fdata[expt][scan]
        func2_diverse(data, f'{expt}_{scan}', samples_per_combination)


NameError: name 'fdata' is not defined